In [ ]:
from random import random, shuffle
import copy
from decimal import Decimal
import time
import copy
import numpy as np

class ConnectSin:
    YOU = 1
    CPU = -1
    EMPTY = 0
    DRAW = 0
    __CONNECT_NUMBER = 4
    board = None

    def __init__(self, board_size=(6, 7), silent=False):
        """
        The main class for the connect4 game

        Inputs
        ----------
        board_size : a tuple representing the board size in format: (rows, columns)
        silent     : whether the game prints outputs or not
        """
        assert len(board_size) == 2, "board size should be a 1*2 tuple"
        assert board_size[0] > 4 and board_size[1] > 4, "board size should be at least 5*5"

        self.columns = board_size[1]
        self.rows = board_size[0]
        self.silent = silent
        self.board_size = self.rows * self.columns

        self.purning_flag = False
        self.max_depth = 1
        self.visited_cell = 0

    def set_max_depth(self, max_d):
        self.max_depth = max_d

    def set_purning_flag(self, flag):
        self.purning_flag = flag

    def get_cell_visited(self):
        return self.visited_cell

    def run(self, starter=None):
        """
        runs the game!

        Inputs
        ----------
        starter : either -1,1 or None. -1 if cpu starts the game, 1 if you start the game. None if you want the starter
            to be assigned randomly

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that the game has drawn
        """
        if (not starter):
            starter = self.__get_random_starter()
        assert starter in [self.YOU, self.CPU], "starter value can only be 1,-1 or None"

        self.__init_board()
        turns_played = 0
        current_player = starter
        while(turns_played < self.board_size):

            if (current_player == self.YOU):
                self.__print_board()
                player_input = self.get_your_input()
            elif (current_player == self.CPU):
                player_input = self.__get_cpu_input()
            else:
                raise Exception("A problem has happend! contact no one, there is no fix!")
            if (not self.register_input(player_input, current_player)):
                self.__print("this move is invalid!")
                continue
            current_player = self.__change_turn(current_player)
            potential_winner = self.check_for_winners()
            turns_played += 1
            if (potential_winner != 0):
                self.__print_board()
                self.__print_winner_message(potential_winner)
                return potential_winner
        self.__print_board()
        self.__print("The game has ended in a draw!")
        return self.DRAW

    def get_your_input(self):
        """
        gets your input

        Output
        ----------
        (int) an integer between 1 and column count. the column to put a piece in
        """
        #TODO: complete here
#         return int(input())

        move, minimax_score = self.minimax(0, Decimal('-Infinity'), Decimal('Infinity'), True)

        return move

    def set_my_score(self, flag):
        if(flag == True):
            return Decimal('-Infinity')
        if(flag == False):
            return Decimal('Infinity')

    def set_my_move(self, moves):
        return moves[0]

    def drop_piece(self, move, current_player):
        last_empty_space = 0
        column_index = move - 1
        for i in range(self.rows):
            if (self.board[i][column_index] == 0):
                last_empty_space = i
        self.board[last_empty_space][column_index] = current_player
        return last_empty_space, column_index

    def minimax(self, depth, alpha, beta, maximizing):
        my_moves = self.get_possible_moves()
        self.visited_cell += 1

        if(len(my_moves) == 0 or self.check_for_winners() != self.EMPTY):
            if(self.check_for_winners() == self.YOU):
                return None, Decimal('Infinity')

            elif(self.check_for_winners() == self.CPU):
                return None, Decimal('-Infinity')

            elif(len(my_moves) == 0):
                return None, 0

        elif(depth == self.max_depth):
            return None, self.my_heuristic()

        my_score = self.set_my_score(maximizing)
        if(self.purning_flag == True):
            shuffle(my_moves)

        my_move = self.set_my_move(my_moves)


        if(maximizing == True):
            for move in my_moves:
                e, idx = self.drop_piece(move, self.YOU)

                score = self.minimax(depth+1, alpha, beta, False)[1]

                self.board[e][idx] = self.EMPTY
                #print("True   ",score, move)
                if(score > my_score):
                    my_score = score
                    my_move = move

                    if(my_score > alpha):
                        alpha = my_score

                    if(self.purning_flag == True):
                        if(alpha >= beta):
                            break

        if(maximizing == False):
            for move in my_moves:
                e, idx = self.drop_piece(move, self.CPU)
                score = self.minimax(depth+1, alpha, beta, True)[1]

                self.board[e][idx] = self.EMPTY
                #print("falsee   ",score)
                if(score < my_score):
                    my_score = score
                    my_move = move

                    if(my_score < beta):
                            beta = my_score

                    if(self.purning_flag == True):
                        if(alpha >= beta):
                            break

        return my_move, my_score

    def check_vertical(self):
        score = 0
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns):
                player_idxs = [0,0,0,0]
                for k in range(self.__CONNECT_NUMBER):
                    if self.board[i + k][j] == self.YOU:
                        player_idxs[k] = 1
                    elif self.board[i + k][j] == self.CPU:
                        player_idxs[k] = -1
                    else:
                        player_idxs[k] = 0

                ss = 0
                #print(player_idxs)
                if player_idxs == [1,1,1,1]:
                 #   print("4v")
                    ss += 100
                elif player_idxs == [0,1,1,1] or player_idxs == [1,0,1,1] or player_idxs == [1,1,0,1] or player_idxs == [1,1,1,0]:
                  #  print("3v")
                    ss += 8
                elif player_idxs == [0,0,1,1] or player_idxs == [1,0,0,1] or player_idxs == [1,1,0,0] or player_idxs == [1,0,1,0] or player_idxs == [0,1,0,1] or player_idxs == [0,1,1,0]:
                   # print("2v")
                    ss += 2
                if player_idxs == [0,-1,-1,-1] or player_idxs == [-1,0,-1,-1] or player_idxs == [-1,-1,0,-1] or player_idxs == [-1,-1,-1,0]:
                    #print("-3v")
                    ss -= 50

                score += ss
        return score

    def check_horizontal(self):
        score = 0
        for i in range(self.rows):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                player_idxs = [0,0,0,0]
                for k in range(self.__CONNECT_NUMBER):
                    if self.board[i][j + k] == self.YOU:
                        player_idxs[k] = 1
                    elif self.board[i][j + k] == self.CPU:
                        player_idxs[k] = -1
                    else:
                        player_idxs[k] = 0

                ss = 0
                #print(player_idxs)
                if player_idxs == [1,1,1,1]:
                    #print("4h")
                    ss += 100
                elif player_idxs == [0,1,1,1] or player_idxs == [1,0,1,1] or player_idxs == [1,1,0,1] or player_idxs == [1,1,1,0]:
                #    print("3h")
                    ss += 8
                elif player_idxs == [0,0,1,1] or player_idxs == [1,0,0,1] or player_idxs == [1,1,0,0] or player_idxs == [1,0,1,0] or player_idxs == [0,1,0,1] or player_idxs == [0,1,1,0]:
                 #   print("2h")
                    ss += 2
                if player_idxs == [0,-1,-1,-1] or player_idxs == [-1,0,-1,-1] or player_idxs == [-1,-1,0,-1] or player_idxs == [-1,-1,-1,0]:
                  #  print("-3h")
                    ss -= 50

                score += ss
        return score

    def check_right_diagonal(self):
        score = 0
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                player_idxs = [0,0,0,0]
                for k in range(self.__CONNECT_NUMBER):
                    if self.board[i + k][j + k] == self.YOU:
                        player_idxs[k] = 1
                    elif self.board[i + k][j + k] == self.CPU:
                        player_idxs[k] = -1
                    else:
                        player_idxs[k] = 0

                ss = 0
                #print(player_idxs)
                if player_idxs == [1,1,1,1]:
                #    print("4rd")
                    ss += 100
                elif player_idxs == [0,1,1,1] or player_idxs == [1,0,1,1] or player_idxs == [1,1,0,1] or player_idxs == [1,1,1,0]:
                 #   print("3rd")
                    ss += 8
                elif player_idxs == [0,0,1,1] or player_idxs == [1,0,0,1] or player_idxs == [1,1,0,0] or player_idxs == [1,0,1,0] or player_idxs == [0,1,0,1] or player_idxs == [0,1,1,0]:
                  #  print("2rd")
                    ss += 2
                if player_idxs == [0,-1,-1,-1] or player_idxs == [-1,0,-1,-1] or player_idxs == [-1,-1,0,-1] or player_idxs == [-1,-1,-1,0]:
                   # print("-3rd")
                    ss -= 50

                score += ss
        return score

    def check_left_diagonal(self):
        score = 0
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                player_idxs = [0,0,0,0]
                for k in range(self.__CONNECT_NUMBER):
                    if self.board[i - k + 3][j + k] == self.YOU:
                        player_idxs[k] = 1
                    elif self.board[i - k + 3][j + k] == self.CPU:
                        player_idxs[k] = -1
                    else:
                        player_idxs[k] = 0

                ss = 0
                #print(player_idxs)
                if player_idxs == [1,1,1,1]:
                #    print("4ld")
                    ss += 100
                elif player_idxs == [0,1,1,1] or player_idxs == [1,0,1,1] or player_idxs == [1,1,0,1] or player_idxs == [1,1,1,0]:
                 #   print("3ld")
                    ss += 8
                elif player_idxs == [0,0,1,1] or player_idxs == [1,0,0,1] or player_idxs == [1,1,0,0] or player_idxs == [1,0,1,0] or player_idxs == [0,1,0,1] or player_idxs == [0,1,1,0]:
                  #  print("2ld")
                    ss += 2
                if player_idxs == [0,-1,-1,-1] or player_idxs == [-1,0,-1,-1] or player_idxs == [-1,-1,0,-1] or player_idxs == [-1,-1,-1,0]:
                   # print("-3ld")
                    ss -= 50

                score += ss
        return score

    def check_diagonal(self):
        score_1 = self.check_right_diagonal()
        score_2 = self.check_left_diagonal()

        return score_1 + score_2

    def my_heuristic(self):

        score_h = self.check_horizontal()
        #print("11 hh", score_h)
        score_v = self.check_vertical()
        #print("11 vv", score_v)
        score_d = self.check_diagonal()
        #print("11 dd", score_d)
        return score_h + score_v + score_d

    def check_for_winners(self):
        """
        checks if anyone has won in this position

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that nothing has happened
        """
        have_you_won = self.check_if_player_has_won(self.YOU)
        if have_you_won:
            return self.YOU
        has_cpu_won = self.check_if_player_has_won(self.CPU)
        if has_cpu_won:
            return self.CPU
        return self.EMPTY

    def check_if_player_has_won(self, player_id):
        """
        checks if player with player_id has won

        Inputs
        ----------
        player_id : the id for the player to check

        Output
        ----------
        (boolean) true if the player has won in this position
        """
        return (
            self.__has_player_won_diagonally(player_id)
            or self.__has_player_won_horizentally(player_id)
            or self.__has_player_won_vertically(player_id)
        )

    def is_move_valid(self, move):
        """
        checks if this move can be played

        Inputs
        ----------
        move : the column to place a piece in, in range [1, column count]

        Output
        ----------
        (boolean) true if the move can be played
        """
        if (move < 1 or move > self.columns):
            return False
        column_index = move - 1
        return self.board[0][column_index] == 0

    def get_possible_moves(self):
        """
        returns a list of possible moves for the next move

        Output
        ----------
        (list) a list of numbers of columns that a piece can be placed in
        """
        possible_moves = []
        for i in range(self.columns):
            move = i + 1
            if (self.is_move_valid(move)):
                possible_moves.append(move)
        return possible_moves

    def register_input(self, player_input, current_player):
        """
        registers move to board, remember that this function changes the board

        Inputs
        ----------
        player_input : the column to place a piece in, in range [1, column count]
        current_player: ID of the current player, either self.YOU or self.CPU

        """
        if (not self.is_move_valid(player_input)):
            return False
        self.__drop_piece_in_column(player_input, current_player)
        return True

    def __init_board(self):
        self.board = []
        for i in range(self.rows):
            self.board.append([self.EMPTY] * self.columns)

    def __print(self, message: str):
        if not self.silent:
            print(message)

    def __has_player_won_horizentally(self, player_id):
        for i in range(self.rows):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_vertically(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_diagonally(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + self.__CONNECT_NUMBER - 1 - x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __get_random_starter(self):
        players = [self.YOU, self.CPU]
        return players[int(random() * len(players))]

    def __get_cpu_input(self):
        """
        This is where clean code goes to die.
        """
        #return int(input())
        bb = copy.deepcopy(self.board)
        pm = self.get_possible_moves()
        for m in pm:
            self.register_input(m, self.CPU)
            if (self.check_if_player_has_won(self.CPU)):
                self.board = bb
                return m
            self.board = copy.deepcopy(bb)
        if (self.is_move_valid((self.columns // 2) + 1)):
            c = 0
            cl = (self.columns // 2) + 1
            for x in range(self.rows):
                if (self.board[x][cl] == self.CPU):
                    c += 1
            if (random() < 0.65):
                return cl
        return pm[int(random() * len(pm))]

    def __drop_piece_in_column(self, move, current_player):
        last_empty_space = 0
        column_index = move - 1
        for i in range(self.rows):
            if (self.board[i][column_index] == 0):
                last_empty_space = i
        self.board[last_empty_space][column_index] = current_player
        return True

    def __print_winner_message(self, winner):
        if (winner == self.YOU):
            self.__print("congrats! you have won!")
        else:
            self.__print("gg. CPU has won!")

    def __change_turn(self, turn):
        if (turn == self.YOU):
            return self.CPU
        else:
            return self.YOU

    def __print_board(self):
        if (self.silent): return
        print("Y: you, C: CPU")
        for i in range(self.rows):
            for j in range(self.columns):
                house_char = "O"
                if (self.board[i][j] == self.YOU):
                    house_char = "Y"
                elif (self.board[i][j] == self.CPU):
                    house_char = "C"

                print(f"{house_char}", end=" ")
            print()


In [ ]:
board_sizes_to_check = [(6,7),
                        (7,8),
                        (7,10)]

game = ConnectSin(board_size=(6,7),silent=False)
game.set_max_depth(1)
game.set_purning_flag(True)
game.run()

Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
11 hh 0
11 vv 0
11 dd 0
True    0 6
11 hh 0
11 vv 0
11 dd 0
True    0 7
11 hh 0
11 vv 0
11 dd 0
True    0 3
11 hh 0
11 vv 0
11 dd 0
True    0 4
11 hh 0
11 vv 0
11 dd 0
True    0 2
11 hh 0
11 vv 0
11 dd 0
True    0 1
11 hh 0
11 vv 0
11 dd 0
True    0 5
7
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O Y C 
11 hh 0
11 vv 0
11 dd 0
True    0 1
11 hh 0
11 vv 0
11 dd 0
True    0 7
11 hh 2
11 vv 0
11 dd 0
True    2 3
11 hh 0
11 vv 0
11 dd 0
True    0 2
11 hh 0
11 vv 2
11 dd 0
True    2 6
11 hh 2
11 vv 0
11 dd 0
True    2 5
11 hh 2
11 vv 0
11 dd 0
True    2 4
6
Y: you, C: CPU
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O O O 
O O O O O C O 
O O Y O O Y C 
11 hh 2
11 vv 2
11 dd 0
True    4 3
11 hh 10
11 vv 0
11 dd 0
True    10 5
11 hh 2
11 vv 0
11 dd 0
True    2 7
11 hh 4
11 vv 0
11 dd 0
True    4 1
11 hh 2
11 vv 0
11 dd 0
True   

1

In [ ]:
win_counter = 0
cell_counter = 0
s = 0
e = 0
for i in range(200):
    game = ConnectSin(board_size=(6, 7),silent=True)
    game.set_max_depth(1)
    game.set_purning_flag(True)
    s += time.time()
    if game.run() == 1:
        win_counter += 1

    e += time.time()

    cell_counter += game.get_cell_visited()

print("using alpha beta purning: with average ",(e - s) / 200, "time taken in depth 1, \nand ", cell_counter / 200, "average visited nodes, wining chance with board 6*7 is: ", win_counter / 2, "%\n")


using alpha beta purning: with average  0.01488037109375 time taken in depth 1, 
and  36.985 average visited nodes, wining chance with board 6*7 is:  96.0 %



In [ ]:
win_counter = 0
cell_counter = 0
s = 0
e = 0
for i in range(200):
    game = ConnectSin(board_size=(7, 8),silent=True)
    game.set_max_depth(1)
    game.set_purning_flag(True)
    s += time.time()
    if game.run() == 1:
        win_counter += 1

    e += time.time()

    cell_counter += game.get_cell_visited()

print("using alpha beta purning: with average ",(e - s) / 200, "time taken in depth 1, \nand ", cell_counter / 200, "average visited nodes, wining chance with board 7*8 is: ", win_counter / 2, "%\n")


using alpha beta purning: with average  0.02449737548828125 time taken in depth 1, 
and  40.945 average visited nodes, wining chance with board 7*8 is:  99.0 %



In [ ]:
win_counter = 0
cell_counter = 0
s = 0
e = 0
for i in range(200):
    game = ConnectSin(board_size=(7, 10),silent=True)
    game.set_max_depth(1)
    game.set_purning_flag(True)
    s += time.time()
    if game.run() == 1:
        win_counter += 1

    e += time.time()

    cell_counter += game.get_cell_visited()

print("using alpha beta purning: with average ",(e - s) / 200, "time taken in depth 1, \nand ", cell_counter / 200, "average visited nodes, wining chance with board 7*10 is: ", win_counter / 2, "%\n")


using alpha beta purning: with average  0.03708465576171875 time taken in depth 1, 
and  45.81 average visited nodes, wining chance with board 7*10 is:  100.0 %



In [ ]:
win_counter = 0
cell_counter = 0
s = 0
e = 0
for i in range(200):
    game = ConnectSin(board_size=(6, 7),silent=True)
    game.set_max_depth(3)
    game.set_purning_flag(True)
    s += time.time()
    if game.run() == 1:
        win_counter += 1

    e += time.time()

    cell_counter += game.get_cell_visited()

print("using alpha beta purning: with average ",(e - s) / 200, "time taken in depth 3, \nand ", cell_counter / 200, "average visited nodes, wining chance with board 6*7 is: ", win_counter / 2, "%\n")


using alpha beta purning: with average  0.23950469970703125 time taken in depth 3, 
and  822.325 average visited nodes, wining chance with board 6*7 is:  100.0 %



In [ ]:
win_counter = 0
cell_counter = 0
s = 0
e = 0
for i in range(200):
    game = ConnectSin(board_size=(7, 8),silent=True)
    game.set_max_depth(3)
    game.set_purning_flag(True)
    s += time.time()
    if game.run() == 1:
        win_counter += 1

    e += time.time()

    cell_counter += game.get_cell_visited()

print("using alpha beta purning: with average ",(e - s) / 200, "time taken in depth 3, \nand ", cell_counter / 200, "average visited nodes, wining chance with board 7*8 is: ", win_counter / 2, "%\n")


using alpha beta purning: with average  0.486488037109375 time taken in depth 3, 
and  1111.625 average visited nodes, wining chance with board 7*8 is:  100.0 %



In [ ]:
win_counter = 0
cell_counter = 0
s = 0
e = 0
for i in range(100):
    game = ConnectSin(board_size=(7, 10),silent=True)
    game.set_max_depth(3)
    game.set_purning_flag(True)
    s += time.time()
    if game.run() == 1:
        win_counter += 1

    e += time.time()

    cell_counter += game.get_cell_visited()

print("using alpha beta purning: with average ",(e - s) / 100, "time taken in depth 3, \nand ", cell_counter / 100, "average visited nodes, wining chance with board 7*10 is: ", win_counter, "%\n")


using alpha beta purning: with average  0.9597573852539063 time taken in depth 3, 
and  1551.5 average visited nodes, wining chance with board 7*10 is:  100 %



In [ ]:
win_counter = 0
cell_counter = 0
s = 0
e = 0
for i in range(200):
    game = ConnectSin(board_size=(6, 7),silent=True)
    game.set_max_depth(5)
    game.set_purning_flag(True)
    s += time.time()
    if game.run() == 1:
        win_counter += 1

    e += time.time()

    cell_counter += game.get_cell_visited()

print("using alpha beta purning: with average ",(e - s) / 20, "time taken in depth 5, \nand ", cell_counter / 20, "average visited nodes, wining chance with board 6*7 is: ", win_counter*10 / 2, "%\n")


using alpha beta purning: with average  3.2841468811035157 time taken in depth 5, 
and  11426.75 average visited nodes, wining chance with board 6*7 is:  100.0 %



In [ ]:
win_counter = 0
cell_counter = 0
s = 0
e = 0
for i in range(200):
    game = ConnectSin(board_size=(7, 8),silent=True)
    game.set_max_depth(5)
    game.set_purning_flag(True)
    s += time.time()
    if game.run() == 1:
        win_counter += 1

    e += time.time()

    cell_counter += game.get_cell_visited()

print("using alpha beta purning: with average ",(e - s) / 20, "time taken in depth 5, \nand ", cell_counter / 20, "average visited nodes, wining chance with board 7*8 is: ", win_counter*10 / 2, "%\n")


using alpha beta purning: with average  6.183507919311523 time taken in depth 5, 
and  14792.0 average visited nodes, wining chance with board 7*8 is:  100.0 %



In [ ]:
win_counter = 0
cell_counter = 0
s = 0
e = 0
for i in range(200):
    game = ConnectSin(board_size=(7, 10),silent=True)
    game.set_max_depth(5)
    game.set_purning_flag(True)
    s += time.time()
    if game.run() == 1:
        win_counter += 1

    e += time.time()

    cell_counter += game.get_cell_visited()

print("using alpha beta purning: with average ",(e - s) / 20, "time taken in depth 5, \nand ", cell_counter / 20, "average visited nodes, wining chance with board 7*10 is: ", win_counter*10 / 2, "%\n")


using alpha beta purning: with average  20.449317359924315 time taken in depth 5, 
and  35581.8 average visited nodes, wining chance with board 7*10 is:  100.0 %



In [ ]:
win_counter = 0
cell_counter = 0
s = 0
e = 0
for i in range(200):
    game = ConnectSin(board_size=(6, 7),silent=True)
    game.set_max_depth(7)
    game.set_purning_flag(True)
    s += time.time()
    if game.run() == 1:
        win_counter += 1

    e += time.time()

    cell_counter += game.get_cell_visited()

print("using alpha beta purning: with average ",(e - s) / 20, "time taken in depth 7, \nand ", cell_counter / 20, "average visited nodes, wining chance with board 6*7 is: ", win_counter*10 / 2, "%\n")


using alpha beta purning: with average  38.18613739013672 time taken in depth 7, 
and  143967.15 average visited nodes, wining chance with board 6*7 is:  100.0 %



In [ ]:
win_counter = 0
cell_counter = 0
s = 0
e = 0
for i in range(200):
    game = ConnectSin(board_size=(7, 8),silent=True)
    game.set_max_depth(7)
    game.set_purning_flag(True)
    s += time.time()
    if game.run() == 1:
        win_counter += 1

    e += time.time()

    cell_counter += game.get_cell_visited()

print("using alpha beta purning: with average ",(e - s) / 10, "time taken in depth 7, \nand ", cell_counter / 10, "average visited nodes, wining chance with board 7*8 is: ", win_counter*10, "%\n")


using alpha beta purning: with average  127.09054260253906 time taken in depth 7, 
and  307312.1 average visited nodes, wining chance with board 7*8 is:  100 %



In [ ]:
win_counter = 0
cell_counter = 0
s = 0
e = 0
for i in range(200):
    game = ConnectSin(board_size=(7, 10),silent=True)
    game.set_max_depth(7)
    game.set_purning_flag(True)
    s += time.time()
    if game.run() == 1:
        win_counter += 1

    e += time.time()

    cell_counter += game.get_cell_visited()

print("using alpha beta purning: with average ",(e - s) / 2, "time taken in depth 7, \nand ", cell_counter / 2, "average visited nodes, wining chance with board 7*10 is: ", win_counter*100 / 2, "%\n")


using alpha beta purning: with average  469.80630922317505 time taken in depth 7, 
and  826729.0 average visited nodes, wining chance with board 7*10 is:  100.0 %

